## Climate time series analysis
* Charge and preprocess data
* Time series clustering 
* Causal analysis

In [44]:
import pandas as pd
import numpy as np
import os
import datetime
from tensorflow.keras import layers, Model
from tslearn.clustering import TimeSeriesKMeans

In [3]:
col_names = pd.read_csv('../dades/col_names.txt', header=None)
df = pd.DataFrame()

df.loc[:,'timestamp'] = pd.read_csv('../dades/data_per_pix/-0.5,-48.5.csv', header=None).loc[:,0].apply(datetime.datetime.fromtimestamp)

In [2]:
len(os.listdir('../dades/data_per_pix/'))

13072

In [7]:
variables_names = ['Lag_RN_ERA_Ix5day_2', 'Lag_SM_GLEAM_ResPlusTrend_Sx5day_2','maxTXx_cum_T_ERA_2', 'monthly_Rx1day_P_CPCU']
variables_ind = [1486, 2109, 4225, 5007]

In [40]:
len(os.listdir('../dades/data_per_pix/'))

13072

In [38]:
df = np.empty((len(os.listdir('../test_data/data_per_pix/')),384,4))
pixels = []
i = 0

for file in os.listdir('../test_data/data_per_pix/'):
    df[i,:,:] = pd.read_csv(os.path.join('../test_data/data_per_pix/', file), header=None).iloc[:, variables_ind]
    i += 1
    pixels.append(file)

In [49]:
inp = layers.Input(shape=(df.shape[1], df.shape[2]))
    

encoder = layers.TimeDistributed(layers.Dense(50, activation='tanh'))(inp)
encoder = layers.TimeDistributed(layers.Dense(10, activation='tanh'))(encoder)
latent = layers.TimeDistributed(layers.Dense(1, activation='tanh'))(encoder)
decoder = layers.TimeDistributed(layers.Dense(10, activation='tanh'))(latent)
decoder = layers.TimeDistributed(layers.Dense(50, activation='tanh'))(decoder)
out = layers.TimeDistributed(layers.Dense(4))(decoder)

autoencoder = Model(inputs=inp, outputs=out)
autoencoder.compile(optimizer='adam', loss='mse')


autoencoder.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 384, 4)]          0         
                                                                 
 time_distributed_18 (TimeDi  (None, 384, 50)          250       
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 384, 10)          510       
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 384, 1)           11        
 stributed)                                                      
                                                                 
 time_distributed_21 (TimeDi  (None, 384, 10)          20        
 stributed)                                                

In [50]:
ind = np.random.permutation(df.shape[0])
training_idx, test_idx = ind[:80], ind[80:]
x_train, x_test = df[training_idx,:,:], df[test_idx,:,:]

In [56]:
history = autoencoder.fit(x_train, x_train, epochs = 2000, validation_data= (x_test, x_test))

Epoch 1/2000
3/3 [==============================] - 0s 83ms/step - loss: 328.4013 - val_loss: 377.4785
Epoch 2/2000
3/3 [==============================] - 0s 16ms/step - loss: 328.3653 - val_loss: 377.4803
Epoch 3/2000
3/3 [==============================] - 0s 16ms/step - loss: 327.7227 - val_loss: 377.3147
Epoch 4/2000
3/3 [==============================] - 0s 16ms/step - loss: 328.1170 - val_loss: 376.4172
Epoch 5/2000
3/3 [==============================] - 0s 16ms/step - loss: 327.3600 - val_loss: 374.4699
Epoch 6/2000
3/3 [==============================] - 0s 16ms/step - loss: 326.1248 - val_loss: 373.8802
Epoch 7/2000
3/3 [==============================] - 0s 16ms/step - loss: 325.3070 - val_loss: 373.4157
Epoch 8/2000
3/3 [==============================] - 0s 16ms/step - loss: 324.7919 - val_loss: 372.7943
Epoch 9/2000
3/3 [==============================] - 0s 16ms/step - loss: 324.3564 - val_loss: 372.5946
Epoch 10/2000
3/3 [==============================] - 0s 16ms/step - loss:

In [57]:
encoder = Model(inputs=inp, outputs=latent)
encoder.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 384, 4)]          0         
                                                                 
 time_distributed_18 (TimeDi  (None, 384, 50)          250       
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 384, 10)          510       
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 384, 1)           11        
 stributed)                                                      
                                                                 
Total params: 771
Trainable params: 771
Non-trainable params: 0
_____________________________________________________________

In [74]:
data = encoder.predict(df)

4/4 [==============================] - 0s 2ms/step


In [75]:
data = data.reshape((len(os.listdir('../test_data/data_per_pix/')), 384))

In [76]:
data = pd.DataFrame(data, index = pixels)

In [78]:
from tslearn.clustering import TimeSeriesKMeans

model = TimeSeriesKMeans(n_clusters=5, metric="dtw", max_iter=100)
model.fit(data)

TimeSeriesKMeans(max_iter=100, metric='dtw', n_clusters=5)

In [79]:
y = model.predict(data)

c:\Users\Usuario\anaconda3\lib\site-packages\tslearn\utils\utils.py:88: UserWarning: 2-Dimensional data passed. Assuming these are 117 1-dimensional timeseries
  warnings.warn('2-Dimensional data passed. Assuming these are '


In [96]:
results = pd.DataFrame()

In [97]:
results['group'] = y


In [98]:
results

,group
0,2
1,0
2,0
3,0
4,0
...,...
112,4
113,4
114,4
115,4


In [100]:
coord_x = []
coord_y = []

for i in range(len(pixels)):
    pixels[i] = pixel.replace('.csv', '')
    coord_x.append(float(pixels[i].split(',')[0]))
    coord_y.append(float(pixels[i].split(',')[1]))

In [102]:
results['coord_x'] = coord_x
results['coord_y'] = coord_y


In [103]:
results

,group,coord_x,coord_y
0,2,67.5,142.5
1,0,67.5,142.5
2,0,67.5,142.5
3,0,67.5,142.5
4,0,67.5,142.5
...,...,...,...
112,4,67.5,142.5
113,4,67.5,142.5
114,4,67.5,142.5
115,4,67.5,142.5
